## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np
import json

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Benghazi,LY,32.12,20.07,77.92,77,0,14.79,clear sky
1,1,Yellowknife,CA,62.46,-114.35,50.00,46,20,6.93,few clouds
2,2,Kamina,CD,-8.74,24.99,66.60,45,0,3.74,clear sky
3,3,Tramore,IE,52.16,-7.15,53.01,83,77,10.00,broken clouds
4,4,Nuuk,GL,64.18,-51.72,48.20,81,100,20.80,light snow
5,5,Hermanus,ZA,-34.42,19.23,54.00,85,100,3.00,overcast clouds
6,6,Victoria,HK,22.29,114.16,88.00,75,58,8.99,broken clouds
7,7,Ushuaia,AR,-54.80,-68.30,37.40,93,90,4.70,light intensity drizzle
8,8,Palana,RU,59.12,159.97,48.99,65,0,10.16,clear sky
9,9,Cockburn Town,TC,21.46,-71.14,82.89,80,98,19.86,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method

cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Benghazi,LY,32.12,20.07,77.92,77,0,14.79,clear sky
6,6,Victoria,HK,22.29,114.16,88.00,75,58,8.99,broken clouds
9,9,Cockburn Town,TC,21.46,-71.14,82.89,80,98,19.86,overcast clouds
12,12,Hasaki,JP,35.73,140.83,95.00,66,20,8.05,few clouds
14,14,Axim,GH,4.87,-2.24,75.27,88,98,8.57,light rain


In [7]:
# 4a. Determine if there are any empty rows
print(len(cities_preferred_df))
cities_preferred_df.count()

240


City_ID                240
City                   240
Country                240
Lat                    240
Lng                    240
Max Temp               240
Humidity               240
Cloudiness             240
Wind Speed             240
Current Description    240
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

240


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Benghazi,LY,32.12,20.07,77.92,77,0,14.79,clear sky
6,6,Victoria,HK,22.29,114.16,88.00,75,58,8.99,broken clouds
9,9,Cockburn Town,TC,21.46,-71.14,82.89,80,98,19.86,overcast clouds
12,12,Hasaki,JP,35.73,140.83,95.00,66,20,8.05,few clouds
14,14,Axim,GH,4.87,-2.24,75.27,88,98,8.57,light rain


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Benghazi,LY,77.92,clear sky,32.12,20.07,
6,Victoria,HK,88.00,broken clouds,22.29,114.16,
9,Cockburn Town,TC,82.89,overcast clouds,21.46,-71.14,
12,Hasaki,JP,95.00,few clouds,35.73,140.83,
14,Axim,GH,75.27,light rain,4.87,-2.24,
17,Georgetown,MY,80.60,scattered clouds,5.41,100.34,
20,Yarada,IN,84.20,mist,17.65,83.27,
23,Siverek,TR,78.80,clear sky,37.76,39.32,
24,Raisinghnagar,IN,84.20,mist,17.65,83.27,
27,Bani Walid,LY,77.92,clear sky,32.12,20.07,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search
    hotels = requests.get(url, params=params).json() 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
    
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [28]:
# 7. Drop the rows where there is no Hotel Name
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Benghazi,LY,77.92,clear sky,32.12,20.07,Tibesti Hotel
6,Victoria,HK,88.00,broken clouds,22.29,114.16,Mini Central
9,Cockburn Town,TC,82.89,overcast clouds,21.46,-71.14,Osprey Beach Hotel
12,Hasaki,JP,95.00,few clouds,35.73,140.83,Inubosaki Kanko Hotel
14,Axim,GH,75.27,light rain,4.87,-2.24,Axim Beach Resort And Spa
17,Georgetown,MY,80.60,scattered clouds,5.41,100.34,Cititel Penang
20,Yarada,IN,84.20,mist,17.65,83.27,Yarada Jungle Beach Resorts
23,Siverek,TR,78.80,clear sky,37.76,39.32,Mercan Hotel
24,Raisinghnagar,IN,84.20,mist,17.65,83.27,Yarada Jungle Beach Resorts
27,Bani Walid,LY,77.92,clear sky,32.12,20.07,Tibesti Hotel


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [3]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

NameError: name 'clean_hotel_df' is not defined

In [31]:
# 11a. Add a marker layer for each city to the map
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))